In [1]:
import utils
import time
import missForest
import category_encoders as ce
import statsmodels.api as sm
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

C:\Users\kogby\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\kogby\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [2]:
# col = [
#     'age',
#     'workclass',
#     'fnlwgt','education','education-num',
#     'marital-status','occupation',
#     'relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','target'
# ]
# df= pd.read_csv("../Example/data/adult/adult.data",names=col)
df = pd.read_csv("../Example/data/HR Employee Attrition.csv")
display(df)
# X = df.drop('target',axis=1)
# Y = df['target']!=' <=50K'
X = df.drop('Attrition',axis=1)
Y = df['Attrition']=='Yes'

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,No,Travel_Frequently,884,Research & Development,23,2,Medical,1,2061,...,3,80,1,17,3,3,5,2,0,3
1466,39,No,Travel_Rarely,613,Research & Development,6,1,Medical,1,2062,...,1,80,1,9,5,3,7,7,1,7
1467,27,No,Travel_Rarely,155,Research & Development,4,3,Life Sciences,1,2064,...,2,80,1,6,0,3,6,2,0,3
1468,49,No,Travel_Frequently,1023,Sales,2,3,Medical,1,2065,...,4,80,0,17,3,2,9,6,0,8


In [16]:
missing_result = utils.simulate_nan(X, nan_rate=0.2)
mis_df = missing_result['X']
display(mis_df)

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Travel_Rarely,1102,Sales,1,NaN,NaN,1,1,NaN,...,1,80,NaN,8,0,1,6,4,0,5
1,49,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,...,4,NaN,1,10,3,3,10,7,1,7
2,NaN,Travel_Rarely,NaN,Research & Development,2,2,Other,NaN,4,4,...,2,80,0,7,NaN,3,0,0,NaN,0
3,33,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,NaN,4,...,3,NaN,0,8,3,3,NaN,7,3,0
4,27,NaN,591,Research & Development,2,1,NaN,1,7,1,...,4,80,1,NaN,3,3,2,2,2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,Travel_Frequently,NaN,NaN,23,2,Medical,NaN,2061,3,...,3,NaN,1,17,NaN,3,NaN,2,NaN,3
1466,39,Travel_Rarely,NaN,Research & Development,6,1,NaN,1,2062,4,...,1,80,1,9,5,3,7,7,NaN,7
1467,NaN,Travel_Rarely,155,Research & Development,NaN,NaN,NaN,NaN,2064,NaN,...,2,80,1,6,0,3,6,2,0,NaN
1468,49,NaN,1023,Sales,2,3,NaN,NaN,2065,4,...,NaN,NaN,0,17,3,2,9,6,NaN,8


In [17]:
encoder = OneHotEncoder(handle_unknown='ignore')
encoder.fit(X,Y)
X_encoded = encoder.transform(X)
display(X_encoded)

X_train, X_test, y_train, y_test = train_test_split(X_encoded, Y, test_size=0.2, random_state=1105)
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
print(f"Score: {xgb.score(X_test,y_test)}")

<1470x5504 sparse matrix of type '<class 'numpy.float64'>'
	with 49980 stored elements in Compressed Sparse Row format>

Score: 0.8843537414965986


In [21]:
encoder = OneHotEncoder(handle_unknown='ignore')
X_encoded = encoder.fit_transform(mis_df)
display(X_encoded)

X_train, X_test, y_train, y_test = train_test_split(X_encoded, Y, test_size=0.2, random_state=1105)
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
print(f"Score: {xgb.score(X_test,y_test)}")

<1470x4590 sparse matrix of type '<class 'numpy.float64'>'
	with 49980 stored elements in Compressed Sparse Row format>

Score: 0.8843537414965986


In [19]:
display(mis_df.head())

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Travel_Rarely,1102,Sales,1,NaN,NaN,1,1,NaN,...,1,80,NaN,8,0,1,6,4,0,5
1,49,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,...,4,NaN,1,10,3,3,10,7,1,7
2,NaN,Travel_Rarely,NaN,Research & Development,2,2,Other,NaN,4,4,...,2,80,0,7,NaN,3,0,0,NaN,0
3,33,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,NaN,4,...,3,NaN,0,8,3,3,NaN,7,3,0
4,27,NaN,591,Research & Development,2,1,NaN,1,7,1,...,4,80,1,NaN,3,3,2,2,2,NaN


In [7]:
mf_imputer = missForest.MissForest(max_iter = 2)
start_time = time.time()
df_imp = mf_imputer.fit_transform(mis_df, verbose=True)
print(f"Execution time: {time.time() - start_time}")

Iteration: 1/2
Continuous: Age 
Using LGBMRegressor(verbosity=-1)
Category: BusinessTravel 
Using LGBMClassifier(verbosity=-1)
Continuous: DailyRate 
Using LGBMRegressor(verbosity=-1)
Category: Department 
Using LGBMClassifier(verbosity=-1)
Continuous: DistanceFromHome 
Using LGBMRegressor(verbosity=-1)
Continuous: Education 
Using LGBMRegressor(verbosity=-1)
Category: EducationField 
Using LGBMClassifier(verbosity=-1)
Continuous: EmployeeCount 
Using LGBMRegressor(verbosity=-1)
Continuous: EmployeeNumber 
Using LGBMRegressor(verbosity=-1)
Continuous: EnvironmentSatisfaction 
Using LGBMRegressor(verbosity=-1)
Category: Gender 
Using LGBMClassifier(verbosity=-1)
Continuous: HourlyRate 
Using LGBMRegressor(verbosity=-1)
Continuous: JobInvolvement 
Using LGBMRegressor(verbosity=-1)
Continuous: JobLevel 
Using LGBMRegressor(verbosity=-1)
Category: JobRole 
Using LGBMClassifier(verbosity=-1)
Continuous: JobSatisfaction 
Using LGBMRegressor(verbosity=-1)
Category: MaritalStatus 
Using LGBMCl

In [8]:
encoder = OneHotEncoder(handle_unknown='ignore')
df_imp_encoded = encoder.fit_transform(df_imp)
display(X_encoded)

X_train, X_test, y_train, y_test = train_test_split(df_imp_encoded, Y, test_size=0.2, random_state=1105)
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
print(f"Score: {xgb.score(X_test,y_test)}")

<1470x4597 sparse matrix of type '<class 'numpy.float64'>'
	with 49980 stored elements in Compressed Sparse Row format>

Score: 0.8673469387755102


In [9]:
display(mis_df)

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,37.229484,Travel_Rarely,687.104750,Sales,1.000000,2.000000,Life Sciences,1.0,1155.846000,2.000000,...,1.000000,80.0,0.000000,8.000000,0.000000,1.0,6.0,4.099480,1.141108,5.0
1,33.599988,Travel_Frequently,279.000000,Research & Development,11.191345,1.000000,Life Sciences,1.0,2.000000,3.000000,...,2.280618,80.0,1.000000,9.665278,3.000000,3.0,10.0,7.000000,1.000000,7.0
2,37.000000,Travel_Rarely,634.481545,Research & Development,2.000000,3.569473,Other,1.0,920.977206,4.000000,...,2.000000,80.0,0.000000,7.000000,2.286668,3.0,0.0,0.073611,0.000000,0.0
3,33.000000,Travel_Frequently,1392.000000,Research & Development,12.359231,4.000000,Life Sciences,1.0,5.000000,4.000000,...,3.000000,80.0,0.000000,6.817560,3.000000,3.0,8.0,7.000000,3.000000,0.0
4,27.000000,Travel_Rarely,591.000000,Research & Development,2.000000,1.000000,Medical,1.0,1211.901931,2.482339,...,4.000000,80.0,1.000000,5.117558,3.000000,3.0,2.0,2.000000,2.000000,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,45.936227,Travel_Frequently,963.241784,Research & Development,23.000000,3.646738,Medical,1.0,2061.000000,3.000000,...,3.000000,80.0,0.970553,17.000000,3.000000,3.0,5.0,2.000000,0.000000,3.0
1466,39.000000,Travel_Rarely,613.000000,Research & Development,6.000000,1.000000,Medical,1.0,2062.000000,4.000000,...,1.000000,80.0,1.000000,9.000000,5.000000,3.0,7.0,7.000000,1.000000,7.0
1467,27.000000,Travel_Rarely,1024.714007,Research & Development,4.000000,2.940042,Life Sciences,1.0,2064.000000,2.000000,...,2.343020,80.0,1.000000,6.000000,0.000000,3.0,6.0,2.000000,0.000000,3.0
1468,49.000000,Travel_Rarely,1023.000000,Sales,2.000000,3.000000,Medical,1.0,2065.000000,4.000000,...,4.000000,80.0,0.000000,17.000000,3.000000,2.0,9.0,6.000000,0.000000,8.0


In [14]:
vtype = np.vectorize(type)

mappings = {}
r_mappings = {}
for col in mis_df.columns:
    feature_without_na = df[col].dropna()
    feature_without_na_type = vtype(feature_without_na)
    print(feature_without_na_type)
    is_all_str = all(feature_without_na_type == str)
    if is_all_str:
        unique_vals = df[col].dropna().unique()
        nunique_vals = range(df[col].dropna().nunique())

        # label the category into int values.
        mappings[col] = {k: v for k, v in zip(unique_vals, nunique_vals)}
        r_mappings[col] = {v: k for k, v in mappings[col].items()}

[<class 'int'> <class 'int'> <class 'int'> ... <class 'int'> <class 'int'>
 <class 'int'>]
[<class 'str'> <class 'str'> <class 'str'> ... <class 'str'> <class 'str'>
 <class 'str'>]
[<class 'int'> <class 'int'> <class 'int'> ... <class 'int'> <class 'int'>
 <class 'int'>]
[<class 'str'> <class 'str'> <class 'str'> ... <class 'str'> <class 'str'>
 <class 'str'>]
[<class 'int'> <class 'int'> <class 'int'> ... <class 'int'> <class 'int'>
 <class 'int'>]
[<class 'int'> <class 'int'> <class 'int'> ... <class 'int'> <class 'int'>
 <class 'int'>]
[<class 'str'> <class 'str'> <class 'str'> ... <class 'str'> <class 'str'>
 <class 'str'>]
[<class 'int'> <class 'int'> <class 'int'> ... <class 'int'> <class 'int'>
 <class 'int'>]
[<class 'int'> <class 'int'> <class 'int'> ... <class 'int'> <class 'int'>
 <class 'int'>]
[<class 'int'> <class 'int'> <class 'int'> ... <class 'int'> <class 'int'>
 <class 'int'>]
[<class 'str'> <class 'str'> <class 'str'> ... <class 'str'> <class 'str'>
 <class 'str'>]